In [72]:
import os
import re
import json
import time
import requests

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from web3 import Web3
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))

def exec(commandString):
    output_stream = os.popen(commandString)
    res = output_stream.read()
    output_stream.close()
    return res

def extractAddress(response, label):
    match = re.findall(re.escape(label) + r"\s+0x[a-fA-F0-9]{40}", response)
    return match[0].split()[-1]

In [45]:
currentTime = int(time.time() * 1000)
daySeconds = 24*3600
dayMSeconds = 1000*daySeconds
START_DATE = 1561687200000;
END_DATE = currentTime  - dayMSeconds - (currentTime%dayMSeconds)
nDAYS = 'max'
AMPL_INITIAL_SUPPLY = 50000000

amplHistroy = json.loads(requests.get('https://web-api.ampleforth.org/eth/token-rebase-history').content)
amplDf = pd.DataFrame(amplHistroy, columns=['epoch', 'price', 'price_target', 'supply', 'time'])
amplDf['time'] = (amplDf['time'] - (amplDf['time'] % daySeconds))
amplDf['time'] = pd.to_datetime(amplDf['time'], unit='s')
amplDf = amplDf.set_index('time')
amplDf['marketcap'] = amplDf['price'] * amplDf['supply']

# NOTE: starting from 17th june 2020
amplDf = amplDf[amplDf.epoch>353]

In [73]:
TRANCHE_RATIOS = [500, 500]

BOND_DURATION_EPOCHS = 24 # number of rebases before the bond ends
ISSUE_FREQUENCY_EPOCHS = 6 # number of rebases before a new bond is issued

TIME_BETWEEN_REBASES = 3600
BOND_DURATION = BOND_DURATION_EPOCHS * TIME_BETWEEN_REBASES
ISSUE_FREQUENCY = ISSUE_FREQUENCY_EPOCHS * TIME_BETWEEN_REBASES

BOND_YIELDS = [1.0, 0]
YIELD_DECIMALS = 18

AMPL_DECIMALS = 9

def deployContracts():
    addrs = { }
    addrs["ampl"] = extractAddress(exec('yarn workspace @ampleforthorg/spot-contracts run hardhat --network ganache deploy:MockAMPL'), "AMPL")
    
    addrs["bondFactory"] = extractAddress(exec('yarn workspace @ampleforthorg/spot-contracts run hardhat --network ganache deploy:BondFactory'), "Bond Factory")

    addrs["issuer"] = extractAddress(exec("""
yarn workspace @ampleforthorg/spot-contracts run hardhat --network ganache deploy:BondIssuer \
  --bond-factory-address "%s" \
  --bond-duration "%s" \
  --issue-frequency "%s" \
  --issue-window-offset "0" \
  --collateral-token-address "%s" \
  --tranche-ratios "%s"
    """ % (addrs["bondFactory"], int(BOND_DURATION), int(ISSUE_FREQUENCY), addrs["ampl"], TRANCHE_RATIOS)), "Bond issuer")
    
    addrs["spot"] = extractAddress(exec("""
yarn workspace @ampleforthorg/spot-contracts run hardhat --network ganache deploy:PerpetualTranche \
  --bond-issuer-address "%s" \
  --collateral-token-address "%s" \
  --name "SPOT" \
  --symbol "SPOT"
    """ % (addrs["issuer"], addrs["ampl"])), "perp")
    
    addrs["router"] = extractAddress(exec("yarn workspace @ampleforthorg/spot-contracts run hardhat --network ganache deploy:Router"), "router")

    return addrs

def setYield(spot, ampl, index, yieldVal):
    exec("""
yarn workspace @ampleforthorg/spot-contracts run hardhat --network ganache deploy:YieldStrategy:setYield \
  --yield-strategy-address "%s" \
  --collateral-token-address "%s" \
  --tranche-ratios "%s" \
  --tranche-index "%s" \
  --tranche-yield "%s"
    """ % (spot.functions.yieldStrategy().call(), ampl.address, TRANCHE_RATIOS, index, yieldVal))

def trancheAndDeposit(router, spot, collateralAmount, walletIdx=0):
    exec("""
yarn workspace @ampleforthorg/spot-contracts run hardhat --network ganache ops:trancheAndDeposit \
  --router-address "%s" \
  --perp-address "%s" \
  --collateral-amount "%s" \
  --from-idx "%s"
    """ % (router.address, spot.address, collateralAmount, walletIdx))

def redeem(router, spot, amount, walletIdx=0):
        exec("""
yarn workspace @ampleforthorg/spot-contracts run hardhat --network ganache ops:redeem \
  --router-address "%s" \
  --perp-address "%s" \
  --amount "%s" \
  --from-idx "%s"
    """ % (router.address, spot.address, amount, walletIdx))

def redeemTranches(issuer, wallet):
        exec("""
yarn workspace @ampleforthorg/spot-contracts run hardhat --network ganache ops:redeemTranches \
  --bond-issuer-address "%s" \
  --from-idx "%s"
    """ % (issuer.address, wallet))

def trancheAndRollover(router, spot, collateralAmount, walletIdx):
    exec("""
yarn workspace @ampleforthorg/spot-contracts run hardhat --network ganache ops:trancheAndRollover \
  --router-address "%s" \
  --perp-address "%s" \
  --collateral-amount "%s" \
  --from-idx "%s"
    """ % (router.address, spot.address, collateralAmount, walletIdx))
    
def advanceClock(timeInSec):
    exec("""yarn workspace @ampleforthorg/spot-contracts run hardhat --network ganache ops:increaseTimeBy %s""" % (timeInSec))

def rebase(ampl, perc):
    exec("""yarn workspace @ampleforthorg/spot-contracts run hardhat --network ganache ops:rebase:MockAMPL --ampl-address %s --rebase-perc %s""" % (ampl.address, perc))

def printInfo(spot):
    print(exec("""
yarn workspace @ampleforthorg/spot-contracts run hardhat --network ganache ops:info "%s"
    """ % (spot.address)))

def toAMPLFixedPt(amt):
    return int(amt * (10 ** AMPL_DECIMALS))

def getBond(bondAddr):
    return w3.eth.contract(address=bondAddr, abi=json.load(open('abis/BondController.json')))

def getCurrentBond(issuer):
    return getBond(issuer.functions.getLatestBond().call())

def getTranche(bond, index):
    return w3.eth.contract(address=bond.functions.tranches(index).call()[0], abi=json.load(open('abis/Tranche.json')))

def getTrancheClass(spot, tranche):
    return w3.toHex(spot.functions.trancheClass(tranche.address).call())


In [74]:
# setup contracts
addrs = deployContracts()
ampl = w3.eth.contract(address=addrs['ampl'], abi=json.load(open('abis/UFragments.json')))
spot = w3.eth.contract(address=addrs['spot'], abi=json.load(open('abis/PerpetualTranche.json')))
issuer = w3.eth.contract(address=addrs['issuer'], abi=json.load(open('abis/BondIssuer.json')))
router = w3.eth.contract(address=addrs['router'], abi=json.load(open('abis/RouterV1.json')))

# setup wallets
MINTER_IDX = 1
TREASURY_IDX = 2
deployer = w3.eth.accounts[0]
minter = w3.eth.accounts[MINTER_IDX]
treasury = w3.eth.accounts[TREASURY_IDX]

# seed wallets with ampl
ampl.functions.transfer(minter, toAMPLFixedPt(15000000)).transact({ "from": deployer })
ampl.functions.transfer(treasury, toAMPLFixedPt(15000000)).transact({ "from": deployer })

# configure AMPL
ampl.functions.setMonetaryPolicy(deployer).transact({"from":deployer})

# configure spot
spot.functions.updateTolerableTrancheMaturity(ISSUE_FREQUENCY, BOND_DURATION).transact({ "from": deployer })
for (i, y) in enumerate(BOND_YIELDS[:-1]):
    setYield(spot, ampl, i, 1.0)

# deployed
print(addrs)

# deposit
INITIAL_DEPOSIT = 1000000
trancheAndDeposit(router, spot, INITIAL_DEPOSIT, MINTER_IDX)

# reserve info
printInfo(spot)

{'ampl': '0xA8dfa6810a690A514e2CcA8c2866437B042CCe74', 'bondFactory': '0x5A3497D694d69045bB5059974F437FAF377E48e4', 'issuer': '0x809C7d6c51216FC8eA9e02CB83b6675189CA4a88', 'spot': '0x2Bb60DB639ab6A3bC2A7C2BB7147aD7c2F55686C', 'router': '0x67Edfd83c4CF229CD8B99f270C3BAD4aCD670d87'}


Error HH305: Unrecognized param --from-idx
For more info go to https://hardhat.org/HH305 or run Hardhat with --show-stack-traces


---------------------------------------------------------------
BondIssuer: 0x809C7d6c51216FC8eA9e02CB83b6675189CA4a88
latestBond: 0x0159dcDa24D745a19448De1F61D56D58bd1680b4
---------------------------------------------------------------
PerpetualTranche: 0x2Bb60DB639ab6A3bC2A7C2BB7147aD7c2F55686C
reserve: 0x2Bb60DB639ab6A3bC2A7C2BB7147aD7c2F55686C
collateralToken 0xA8dfa6810a690A514e2CcA8c2866437B042CCe74
feeStrategy: 0xE4bC48cA6DF6919eF3dC3D783063B5d2930b88A1
yieldStrategy: 0x0779f27490Ee1Ed55995f7cDa60620dba4421014
feeToken: 0x2Bb60DB639ab6A3bC2A7C2BB7147aD7c2F55686C
pricingStrategy: 0x0a3a0336Ead8D325085757EAB479Ddc5DA60AfFb
maturityTolarance: [21600, 86400]
depositBond: 0x0159dcDa24D745a19448De1F61D56D58bd1680b4
issued: true
TotalSupply: 0.0
MatureTrancheBalance: 0.0
---------------------------------------------------------------
Reserve:
┌─────────┬─────────┬─────────────┬───────┬───────────────┐
│ (index) │ balance │ yieldFactor │ price │ upForRollover │
├─────────┼─────────┼───

In [49]:

supplies = []
minterAmplBalances = []
minterSpotBalances = []
treasuryAmplBalances = []

for (i, s) in enumerate(amplDf['supply']):    
    ampl.functions.rebase(i, s * (10 ** AMPL_DECIMALS)).transact({ "from": deployer })    
    
    supply = ampl.functions.totalSupply().call()
    supplies.append(supply / (10 ** AMPL_DECIMALS))
    
    minterAmplBalances.append(ampl.functions.balanceOf(minter).call() / (10 ** AMPL_DECIMALS))
    minterSpotBalances.append(spot.functions.balanceOf(minter).call() / (10 ** AMPL_DECIMALS))
    treasuryAmplBalances.append(ampl.functions.balanceOf(treasury).call() / (10 ** AMPL_DECIMALS))
    
    // rollover
    // redeem
        
    advanceClock(TIME_BETWEEN_REBASES)
    
    # advance time
    if (i % ISSUE_FREQUENCY_EPOCHS == 0):
        spot.functions.updateState().transact({ "from": deployer })

        trancheAndDeposit(router, spot, MINT_PER_ISSUE*2, MINTER_IDX)
        redeem(router, spot, REDEEM_PER_ISSUE, MINTER_IDX)
        print(spot.functions.balanceOf(minter).call())

    if i >= BOND_DURATION_EPOCHS:
        trancheAndRollover(router, spot, MINT_PER_ISSUE*2, TREASURY_IDX)
        
    redeemTranches(issuer, MINTER_IDX)
    redeemTranches(issuer, TREASURY_IDX)

# printInfo(spot)

time
2020-06-18    0.000000
2020-06-19    0.000000
2020-06-20    0.000000
2020-06-21    0.000000
2020-06-22    0.000000
                ...   
2022-07-21    0.004329
2022-07-22    0.000000
2022-07-23    0.000000
2022-07-24    0.000000
2022-07-25    0.008643
Name: supply, Length: 767, dtype: float64

In [71]:
# plt.plot(minterAmplBalances, label="minter-ampl")
# plt.plot(minterSpotBalances, label="minter-spot")
# plt.plot(treasuryAmplBalances, label="treasury-ampl")
# plt.show()

# trancheAndDeposit(router, spot, MINT_PER_ISSUE*2, 0)
# spot.functions.balanceOf(minter).call()


yarn workspace @ampleforthorg/spot-contracts run hardhat --network ganache deploy:YieldStrategy:setYield   --yield-strategy-address "0x4A2b21337Ad262B5959228D35e6ceB6b2796f326"   --collateral-token-address "0xA2A39a5bb9c6328aE93482B515EF9f4845e90C7b"   --tranche-ratios "[500, 500]"   --tranche-index "0"   --tranche-yield "1.0"
    
{
  yieldStrategyAddress: '0x4A2b21337Ad262B5959228D35e6ceB6b2796f326',
  collateralTokenAddress: '0xA2A39a5bb9c6328aE93482B515EF9f4845e90C7b',
  trancheRatios: [ 500, 500 ],
  trancheIndex: '0',
  trancheYield: '1.0'
} 1.0
18
0x5de99950fa01c6ba2c66b200ccc6463f156cc9905da935cf15f3e2a3a309db1a

